In [1]:
import pandas as pd
import numpy as np
import os, glob
import datetime
import bisect
import math
import xlsxwriter
import matplotlib.pyplot as plt
from geopy.distance import geodesic

prgname = 'rsrp_gridwise_cv_mul.ipynb'

In [2]:
path = "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/"
all_files = glob.glob(os.path.join(path, "gps_cell*.csv"))
print(len(all_files))
nfls = len(all_files)

71


In [3]:
#pick the voluteer name
name2 = all_files[0].split("/")[-2]
name = name2.split("_")[-1]
print(name)

Kishore


#### picking all files (route= A or B) and arranging datewise

In [4]:
sroute = "ROUTE_B"                              #change ROUTE A/B
#filese reading from folder
route_files = []
for fle in all_files:
    df = pd.read_csv(fle,low_memory=True)
    numcols = len(df.columns)
    rte = sroute                             
    chk = df['route'].iloc[0]
    if(chk == rte):
        route_files.append(fle)
        
print(len(route_files))
#collecting the date from each file at selected route
dates = []
for fle in route_files:
    #print(fle)
    date1 = fle.split("/")[-1]
    seldate = date1[28:38]
    dates.append(seldate)
    
print(dates)
#sorting the dates and corresponding filenames are copied into array
dates_sort = [dates.index(x) for x in sorted(dates)]
dt_sort = np.asarray(dates_sort)
sorted_fls = np.empty((len(dates)), dtype=object)
for ii in range(len(route_files)):
    jj = dt_sort[ii]
    sorted_fls[ii] = route_files[jj]
    print(ii,jj,dt_sort[ii],route_files[jj])

22
['2023-06-20', '2023-07-18', '2023-07-25', '2023-07-19', '2023-06-21', '2023-08-10', '2023-07-10', '2023-07-06', '2023-07-05', '2023-06-27', '2023-06-28', '2023-08-09', '2023-08-14', '2023-08-15', '2023-06-29', '2023-07-27', '2023-07-26', '2023-06-26', '2023-08-16', '2023-06-22', '2023-06-15', '2023-07-11']
0 20 20 /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-15-16-18-20-428000000-fill_out_your_user_id.csv
1 0 0 /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-20-16-35-05-002000000-fill_out_your_user_id.csv
2 4 4 /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-21-16-59-34-977000000-fill_out_your_user_id.csv
3 19 19 /local/mnt3/workspace3/Kishore/ML

In [5]:
cnt=0; ii=-1
dates = []
list_pnts = []
flon = np.zeros((nfls,6500)) * np.nan
flat = np.zeros((nfls,6500)) * np.nan
frsrp = np.zeros((nfls,6500)) * np.nan
distt = np.zeros((nfls,6500))*np.nan
cdist = np.zeros((nfls,6500))*np.nan
for fle in sorted_fls:
    df = pd.read_csv(fle,low_memory=True)
    numcols = len(df.columns)
    #print("fname: {0}; no. of cloumns: {1}".format(fle, numcols))
    #display(df)
    #display(df.columns)
    rte = sroute
    chk = df['route'].iloc[0]
    if(chk == rte):
        cnt += 1
        print("fname: {0}; no. of rows: {1}".format(fle, len(df)))
        list_pnts.append(len(df))
        date1 = fle.split("/")[-1]
        seldate = date1[28:38]
        dates.append(seldate)
        
        df.drop(["index","GPS","cputime","speed","altitude","index.1",
                "route","cellinfo","cputime.1","ncells","type","rsrq",
                "sinr","rssi","level","dcpu"], axis=1, inplace=True)
        #display(df)
        
        lonn = df['Lon'].astype(float)
        latt = df['Lat'].astype(float)
        values = df['rsrp'].astype(float)
        np = len(lonn)
        ii += 1
        #display(np, ii, cnt)
        #distt = [0.0] * np
        #cdist = [0.0] * np
        cumdist = 0.0
        for kk in range(1,np):
            stpnt = (latt[kk-1],lonn[kk-1])
            endpnt = (latt[kk],lonn[kk])
            #geodesic distance calc. the distance on the surface of an ellipsoidal model of the earth (Karney 2013)
            distt[ii,kk-1] = geodesic(stpnt, endpnt).miles
            cumdist += distt[ii,kk-1]
            cdist[ii,kk-1] = round(cumdist, 3)
            flon[ii,kk-1] = lonn[kk-1]
            flat[ii,kk-1] = latt[kk-1]
            frsrp[ii,kk-1] = values[kk-1]
            #print(kk,latt[kk-1],lonn[kk-1],'   ',distt[ii,kk-1],'  ',cumdist)

print(f"Volunteer: {name}   route: {rte}   tot files: {cnt}")
rfls = (ii+1)
# rfls = 2
flon = flon[0:rfls,:]; flat = flat[0:rfls,:]
distt = distt[0:rfls,:]; cdist = cdist[0:rfls,:]
frsrp = frsrp[0:rfls,:]
print(flon.shape)

fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-15-16-18-20-428000000-fill_out_your_user_id.csv; no. of rows: 4490
fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-20-16-35-05-002000000-fill_out_your_user_id.csv; no. of rows: 4209
fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-21-16-59-34-977000000-fill_out_your_user_id.csv; no. of rows: 3962
fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/signal_predicion_data/gps_cell_mergedata/SP_Kishore/gps_cell_merge-null-unknown-2023-06-22-16-01-11-551000000-fill_out_your_user_id.csv; no. of rows: 6221
fname: /local/mnt3/workspace3/Kishore/ML_project_dir/ML_proj

In [6]:
#finding the minimum and maximum vals of longitude and latitudes
import numpy as np
lonmin=[]; lonmax=[]
latmin=[]; latmax=[]
for ii in range(rfls):
    londat = flon[ii,:]
    latdat = flat[ii,:]
    londata = londat[~np.isnan(londat)]
    latdata = latdat[~np.isnan(latdat)]
    minlon = min(londata); maxlon=max(londata)
    minlat = min(latdata); maxlat = max(latdata)
    lonmin.append(minlon); lonmax.append(maxlon)
    latmin.append(minlat); latmax.append(maxlat)

minlong = min(lonmin); maxlong = max(lonmax)
minlatt = min(latmin); maxlatt = max(latmax)
print(f"min_longitude: {minlong}    max_longitude: {maxlong}")
print(f"min_latitude: {minlatt}    max_latitude: {maxlatt}")

min_longitude: -117.67478826    max_longitude: -117.1930932
min_latitude: 32.89198382    max_latitude: 33.6120343


In [7]:
#fixing the grids (0.1 mile = 40 x 40)
nx = 50
ny = 50
deltax = (maxlong - minlong)/nx
deltay = (maxlatt - minlatt)/ny
print(deltax,deltay)

0.009633901199999855 0.014401009599999952


In [8]:
#distance calcs. using lon, lats grids
slat = minlatt
for clms in range(3):
    slon = minlong
    elat = slat + deltay
    for rws in range(3):
        elon = slon + deltax
        #print(clms,rws,slon,elon, ' ',elon-slon,' ',slat,elat,elat-slat)
        stpnt = (slat,slon)
        endpnt = (elat,elon)
        diss = geodesic(stpnt,endpnt).miles
        print('distance: ',diss)
        
        slon = elon
    slat = elat


distance:  1.1395307368455454
distance:  1.1395307368455454
distance:  1.1395307368455454
distance:  1.1394881785977409
distance:  1.1394881785977409
distance:  1.1394881785977409
distance:  1.1394456090321228
distance:  1.1394456090321228
distance:  1.1394456090321228


In [9]:
sigavg = np.zeros((rfls,nx*ny)) * np.nan
sigstd = np.zeros((rfls,nx*ny)) * np.nan
sigcv = np.zeros((rfls,nx*ny)) * np.nan
glon = np.zeros((rfls,nx*ny))*np.nan
glat = np.zeros((rfls,nx*ny))*np.nan

tcnt = 0
for nf in range(rfls):
    #picking the each grid RSRP values: count, RSRP mean, predictions at each grid
    plon = flon[nf,:]
    plat = flat[nf,:]
    prsrp = frsrp[nf,:]
    #picking the non NaN indices values
    nindxs = np.where(~np.isnan(plon))
    lonp = plon[nindxs]
    latp = plat[nindxs]
    rsrpp = prsrp[nindxs]
    
    slat = minlatt; jj = -1; 
    for clms in range(ny):
        slon = minlong
        elat = slat + deltay
        for rws in range(nx):
            elon = slon + deltax
            cnt = 0; totrsrp = 0.0
            jj+=1; rsrpvals = []
            for kk in range(len(rsrpp)):
                if(((lonp[kk]>=slon) & (lonp[kk]<elon)) & ((latp[kk]>=slat) & (latp[kk]<elat))):
                    #cnt += 1
                    if((rsrpp[kk] >= -156.0) & (rsrpp[kk] <= -31.0) & (~np.isnan(rsrpp[kk]))):
                        cnt += 1
                        rsrpvals.append(rsrpp[kk])
                        totrsrp = totrsrp + rsrpp[kk]
                        
            if(len(rsrpvals) > 0):
                avgsig = np.mean(rsrpvals)
                stdsig = np.std(rsrpvals)
                cvsig = stdsig/avgsig
            else:
                avgsig = np.nan
                stdsig = np.nan
                cvsig = np.nan
                
            glon[nf,jj] = (slon + elon)/2
            glat[nf,jj] = (slat + elat)/2
            sigavg[nf,jj] = avgsig
            sigstd[nf,jj] = stdsig
            sigcv[nf,jj] = cvsig
            #if(avgsig < 0.0): print(nf,clms,rws,avgsig,stdsig,cvsig)
    
            slon = elon
        slat = elat
    print(f"{nf} volunteer: {name}, route: {rte}, date: {dates[nf]} completed")

0 volunteer: Kishore, route: ROUTE_B, date: 2023-06-15 completed
1 volunteer: Kishore, route: ROUTE_B, date: 2023-06-20 completed
2 volunteer: Kishore, route: ROUTE_B, date: 2023-06-21 completed
3 volunteer: Kishore, route: ROUTE_B, date: 2023-06-22 completed
4 volunteer: Kishore, route: ROUTE_B, date: 2023-06-26 completed
5 volunteer: Kishore, route: ROUTE_B, date: 2023-06-27 completed
6 volunteer: Kishore, route: ROUTE_B, date: 2023-06-28 completed
7 volunteer: Kishore, route: ROUTE_B, date: 2023-06-29 completed
8 volunteer: Kishore, route: ROUTE_B, date: 2023-07-05 completed
9 volunteer: Kishore, route: ROUTE_B, date: 2023-07-06 completed
10 volunteer: Kishore, route: ROUTE_B, date: 2023-07-10 completed
11 volunteer: Kishore, route: ROUTE_B, date: 2023-07-11 completed
12 volunteer: Kishore, route: ROUTE_B, date: 2023-07-18 completed
13 volunteer: Kishore, route: ROUTE_B, date: 2023-07-19 completed
14 volunteer: Kishore, route: ROUTE_B, date: 2023-07-25 completed
15 volunteer: Kishor

In [ ]:

for jj in range(rfls):
    fig,ax = plt.subplots(figsize=(20,14))
    ax.set_facecolor('white')
    
    rlon = glon[jj,:]; rlat = glat[jj,:]
    cvar = sigcv[jj,:]
    
    tdd = np.where(~np.isnan(cvar))
    rlonn = rlon[tdd]
    rlatt = rlat[tdd]
    
    sc = ax.scatter(rlon, rlat, c=cvar, cmap='jet',  s=0., alpha=0.65, linewidths=6)
    sc = ax.scatter(df['Lon'], df['Lat'], c=df['rsrp'], cmap='jet',  s=1.5, alpha=0.04, linewidths=6)
    
        
    for ij in range(len(cvar)):
        if(~np.isnan(cvar[ij])):
            data = (round(cvar[ij], 2))
            if(abs(data*100.) <=9):
                ax.text(rlon[ij]-0.0005, rlat[ij]-0.0005,int(round(data*100,0)),fontsize=10,weight='bold',color='blue')
            else:
                ax.text(rlon[ij]-0.0005, rlat[ij]-0.0005,int(round(data*100,0)),fontsize=10,weight='bold',color='red') 
                
    for ii in range(ny+1):
        yval = minlatt + deltay*ii
        ax.hlines(y=yval,xmin=minlong, xmax=maxlong,linewidth=1,color='gray',linestyles='--',alpha=0.45)
    for ii in range(nx+1):
        xval = minlong + deltax*ii
        ax.vlines(x=xval,ymin=minlatt-.001, ymax=maxlatt,linewidth=1,color='gray',linestyles='--',alpha=0.45)
        
    tit = name+ '  '+rte +'  '+dates[jj]
    ax.set_title(tit,fontsize=14,weight='bold')
    plt.annotate(prgname, xy=(0.08,0.02), xycoords='figure fraction', ha='left',color='black',fontsize=8)
    
    dist = float("{:.2f}".format(diss))
    #save image
    outpath= "/local/mnt3/workspace3/Kishore/ML_project_dir/ML_project_env/data_folder/test/signal_prediction_plots/16Aug2023/"
    vfname = outpath+name +'_'+rte+'_'+str(nx)+'x'+str(ny)+'_'+str(dist)+'_'+dates[jj]+ '_RSRP_cell_gps_gridwise_cvs.png'
    #print(vfname)
    fig.savefig(vfname)
    plt.close()